In [ ]:
import cv2
import os
import csv
import mediapipe as mp
import pandas as pd

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.3)

data_base_path = 'Dataset\Five'


# Get all image paths in the dataset folder
your_dataset_paths = [os.path.join(data_base_path, file) for file in os.listdir(data_base_path) if file.endswith(('.png', '.jpg', '.jpeg'))]

# Print all image paths for debugging
#print("Image Paths:", your_dataset_paths)

# Open a CSV file to save the landmarks
with open("landmarks_5.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # Define the header
    header = ["Image Path"]
    # Append "x", "y", "z" for each of the 21 landmarks
    for i in range(21):  # MediaPipe Hands has 21 landmarks
        header.extend([f"x_{i+1}", f"y_{i+1}", f"z_{i+1}"])
    writer.writerow(header)

    # Process each image and write landmarks to the CSV
    for image_path in your_dataset_paths:
        image = cv2.imread(image_path)

        # Check if the image is read correctly
        if image is None:
            print(f"Failed to read image: {image_path}")
            continue  # Skip this image if it couldn't be read

        print(f"Processing image: {image_path}, shape: {image.shape}")
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = hands.process(image_rgb)

        # Start with the image path
        landmarks = [image_path]

        if result.multi_hand_landmarks:
            hand_landmarks = result.multi_hand_landmarks[0]
            # Collect (x, y, z) for each landmark
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            print(f"Processed: {image_path}, Landmarks detected")
        else:
            # If no landmarks detected, fill with None
            landmarks.extend([None] * (3 * 21))  # 21 landmarks with (x, y, z)
            print(f"Processed: {image_path}, No landmarks detected")

        # Write the complete landmarks list for the current image to the CSV
        writer.writerow(landmarks)

print("Landmarks have been saved to landmarks.csv")


In [ ]:
file_paths = [
    "Land marks from each sign\landmarks_1.csv",
    "Land marks from each sign\landmarks_2.csv",
    "Land marks from each sign\landmarks_3.csv",
    "Land marks from each sign\landmarks_4.csv",
    "Land marks from each sign\landmarks_5.csv",
    "Land marks from each sign\landmarks_6.csv",
    "Land marks from each sign\landmarks_7.csv",
    "Land marks from each sign\landmarks_8.csv",
    "Land marks from each sign\landmarks_9.csv",
    "Land marks from each sign\landmarks_10.csv",
    "Land marks from each sign\landmarks_11.csv",
    "Land marks from each sign\landmarks_12.csv",
]


# Dictionary mapping filenames to gesture labels
gesture_labels = {
    "landmarks_1.csv": 1,
    "landmarks_2.csv": 2,
    "landmarks_3.csv": 3,
    "landmarks_4.csv": 4,
    "landmarks_5.csv": 5,
    "landmarks_6.csv": 6,
    "landmarks_7.csv": 7,
    "landmarks_8.csv": 8,
    "landmarks_9.csv": 9,
    "landmarks_10.csv": 10,
    "landmarks_11.csv": 11,
    "landmarks_12.csv": 12
}

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate over the files, read them, and add a 'target' column with the gesture label
for file in file_paths:
    df = pd.read_csv(file)
    gesture_label = gesture_labels.get(file.split("\\")[-1], None)  # Get the gesture label from the dictionary
    df['target'] = gesture_label  # Add the gesture label as the 'target' column
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_data = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
output_path = "combined_land_marks_with_target.csv"
combined_data.to_csv(output_path, index=False)

print(f"Combined CSV with target column saved to: {output_path}")


In [ ]:

# Function to check and remove rows with null values in a CSV file and drop the 'image' column
def check_and_remove_null_values(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check for null values in the DataFrame
    null_values = df.isnull().sum()
    
    print(f"Null values in each column of {file_path}:")
    print(null_values)
    
    # Remove rows with any null values
    df_cleaned = df.dropna()

    # Drop the 'image' column if it exists
    if 'Image Path' in df_cleaned.columns:
        df_cleaned = df_cleaned.drop(columns=['Image Path'])
        print("'Image' column dropped.")
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_file_path = file_path.replace(".csv", "_cleaned.csv")
    df_cleaned.to_csv(cleaned_file_path, index=False)
    
    print(f"Cleaned CSV saved to: {cleaned_file_path}")
    return df_cleaned

# Example usage
# Call the function for the combined CSV file
combined_csv_path = "combined_land_marks_with_target.csv"
check_and_remove_null_values(combined_csv_path)
